# Introdução

In [ ]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Join Data

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_cetesb/main/data/tabs/tab_municipio_cetesb.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

<br>

## Create pop up's column

In [ ]:
# Add Field
def popup_html(row):
    tel_01 = str(row['coord_regional_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_02 = str(row['coord_regional_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_11 = str(row['coord_adj1_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_12 = str(row['coord_adj1_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_21 = str(row['coord_adj2_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_22 = str(row['coord_adj2_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_31 = str(row['coord_adj3_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_32 = str(row['coord_adj3_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_41 = str(row['coord_adj4_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_42 = str(row['coord_adj4_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_51 = str(row['coord_adj5_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_52 = str(row['coord_adj5_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_61 = str(row['coord_adj6_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_62 = str(row['coord_adj6_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_71 = str(row['coord_adj7_tel01']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_72 = str(row['coord_adj7_tel02']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    # 
    tel_11 = tel_11 if pd.isnull(row['coord_adj1_tel01ramal']) else '{}p{}'.format(tel_11, int(row['coord_adj1_tel01ramal']))
    
    
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4>REPDEC de <b>{}</b></h4></p>
    <p><b>Coordenador Regional:</b><br>
    {}<br>
    {}{}<br>
    {}{}<br>
    <br>
    <a id="seeMore" onclick="toggleSeeMore()" href="javascript:void(0);">Coordenadores Adjuntos</a><p>
    <p id="textarea" style="display: none">
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    
    {}
    {}{}
    {}{}
    </p>    
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome'])   else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['repdec_regiao'])   else '{}'.format(row['repdec_regiao']),
        
        '' if pd.isnull(row['coord_regional_nome'])   else '{}'.format(row['coord_regional_nome']),
        '' if pd.isnull(row['coord_regional_mail01']) else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_regional_mail01'], row['coord_regional_mail01']),
        '' if pd.isnull(row['coord_regional_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_regional_mail02'], row['coord_regional_mail02']),
        '' if pd.isnull(row['coord_regional_tel01'])  else 'Telefone: <a href="tel:{}">{}</a>'.format(tel_01, row['coord_regional_tel01']),
        '' if pd.isnull(row['coord_regional_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_02, row['coord_regional_tel02']),
        
        '' if pd.isnull(row['coord_adj1_nome'])   else '<b>Coordenador Adjunto #01</b><br>{}'.format(row['coord_adj1_nome']),
        '' if pd.isnull(row['coord_adj1_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj1_mail01'], row['coord_adj1_mail01']),
        '' if pd.isnull(row['coord_adj1_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj1_mail02'], row['coord_adj1_mail02']),
        '' if pd.isnull(row['coord_adj1_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_11, row['coord_adj1_tel01']),
        '' if pd.isnull(row['coord_adj1_tel01ramal'])  else ' Ramal: {}</a>'.format(int(row['coord_adj1_tel01ramal'])),
        '' if pd.isnull(row['coord_adj1_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_12, row['coord_adj1_tel02']),
        
        '' if pd.isnull(row['coord_adj2_nome'])   else '<br><br><b>Coordenador Adjunto #02</b><br>{}'.format(row['coord_adj2_nome']),
        '' if pd.isnull(row['coord_adj2_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj2_mail01'], row['coord_adj2_mail01']),
        '' if pd.isnull(row['coord_adj2_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj2_mail02'], row['coord_adj2_mail02']),
        '' if pd.isnull(row['coord_adj2_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_21, row['coord_adj2_tel01']),
        '' if pd.isnull(row['coord_adj2_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_22, row['coord_adj2_tel02']),
        
        '' if pd.isnull(row['coord_adj3_nome'])   else '<br><br><b>Coordenador Adjunto #03</b><br>{}'.format(row['coord_adj3_nome']),
        '' if pd.isnull(row['coord_adj3_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj3_mail01'], row['coord_adj3_mail01']),
        '' if pd.isnull(row['coord_adj3_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj3_mail02'], row['coord_adj3_mail02']),
        '' if pd.isnull(row['coord_adj3_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_31, row['coord_adj3_tel01']),
        '' if pd.isnull(row['coord_adj3_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_32, row['coord_adj3_tel02']),
        
        '' if pd.isnull(row['coord_adj4_nome'])   else '<br><br><b>Coordenador Adjunto #04</b><br>{}'.format(row['coord_adj4_nome']),
        '' if pd.isnull(row['coord_adj4_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj4_mail01'], row['coord_adj4_mail01']),
        '' if pd.isnull(row['coord_adj4_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj4_mail02'], row['coord_adj4_mail02']),
        '' if pd.isnull(row['coord_adj4_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_41, row['coord_adj4_tel01']),
        '' if pd.isnull(row['coord_adj4_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_42, row['coord_adj4_tel02']),
        
        '' if pd.isnull(row['coord_adj5_nome'])   else '<br><br><b>Coordenador Adjunto #05</b><br>{}'.format(row['coord_adj5_nome']),
        '' if pd.isnull(row['coord_adj5_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj5_mail01'], row['coord_adj5_mail01']),
        '' if pd.isnull(row['coord_adj5_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj5_mail02'], row['coord_adj5_mail02']),
        '' if pd.isnull(row['coord_adj5_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_51, row['coord_adj5_tel01']),
        '' if pd.isnull(row['coord_adj5_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_52, row['coord_adj5_tel02']),
        
        '' if pd.isnull(row['coord_adj6_nome'])   else '<br><br><b>Coordenador Adjunto #06</b><br>{}'.format(row['coord_adj6_nome']),
        '' if pd.isnull(row['coord_adj6_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj6_mail01'], row['coord_adj6_mail01']),
        '' if pd.isnull(row['coord_adj6_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj6_mail02'], row['coord_adj6_mail02']),
        '' if pd.isnull(row['coord_adj6_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_61, row['coord_adj6_tel01']),
        '' if pd.isnull(row['coord_adj6_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_62, row['coord_adj6_tel02']),
        
        '' if pd.isnull(row['coord_adj7_nome'])   else '<br><br><b>Coordenador Adjunto #07</b><br>{}'.format(row['coord_adj7_nome']),
        '' if pd.isnull(row['coord_adj7_mail01']) else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_adj7_mail01'], row['coord_adj7_mail01']),
        '' if pd.isnull(row['coord_adj7_mail02']) else ' / <a href="mailto:{}">{}</a>'.format(row['coord_adj7_mail02'], row['coord_adj7_mail02']),
        '' if pd.isnull(row['coord_adj7_tel01'])  else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel_71, row['coord_adj7_tel01']),
        '' if pd.isnull(row['coord_adj7_tel02'])  else ' / <a href="tel:{}">{}</a>'.format(tel_72, row['coord_adj7_tel02']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
# Calculate PopUps
gdf['popup'] = gdf.apply(popup_html, axis=1)

<br>

## Adjust Table

In [ ]:
# Delete Columns
gdf.drop([
    'id_municipio', 'repdec_nome', 
    'coord_regional_nome', 'coord_regional_mail01', 'coord_regional_mail02', 'coord_regional_tel01', 'coord_regional_tel02',
    'coord_regional_tel01ramal',  'coord_regional_tel02ramal',    
    'coord_adj1_nome', 'coord_adj1_mail01', 'coord_adj1_mail02', 'coord_adj1_tel01', 'coord_adj1_tel02', 'coord_adj1_tel01ramal', 'coord_adj1_tel02ramal',
    'coord_adj2_nome', 'coord_adj2_mail01', 'coord_adj2_mail02', 'coord_adj2_tel01', 'coord_adj2_tel02', 'coord_adj2_tel01ramal', 'coord_adj2_tel02ramal',
    'coord_adj3_nome', 'coord_adj3_mail01', 'coord_adj3_mail02', 'coord_adj3_tel01', 'coord_adj3_tel02', 'coord_adj3_tel01ramal', 'coord_adj3_tel02ramal',
    'coord_adj4_nome', 'coord_adj4_mail01', 'coord_adj4_mail02', 'coord_adj4_tel01', 'coord_adj4_tel02', 'coord_adj4_tel01ramal', 'coord_adj4_tel02ramal',
    'coord_adj5_nome', 'coord_adj5_mail01', 'coord_adj5_mail02', 'coord_adj5_tel01', 'coord_adj5_tel02', 'coord_adj5_tel01ramal', 'coord_adj5_tel02ramal',
    'coord_adj6_nome', 'coord_adj6_mail01', 'coord_adj6_mail02', 'coord_adj6_tel01', 'coord_adj6_tel02', 'coord_adj6_tel01ramal', 'coord_adj6_tel02ramal',
    'coord_adj7_nome', 'coord_adj7_mail01', 'coord_adj7_mail02', 'coord_adj7_tel01', 'coord_adj7_tel02', 'coord_adj7_tel01ramal', 'coord_adj7_tel02ramal',
], axis=1, inplace=True)
print(gdf.columns)

# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_cetesb.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

## Get Centroid

In [ ]:
def get_centroid(gdf):
    gdf['apenasparacentroid'] = 35
    gdf_dissolve = gdf.dissolve(by='apenasparacentroid')
    gdf_centroid = gdf_dissolve.representative_point()
    gdf = gdf.drop('apenasparacentroid', axis=1)
    return [float(gdf_centroid.y), float(gdf_centroid.x)]

In [ ]:
list_centroid = get_centroid(gdf)
list_centroid

## Folium

In [ ]:
def map_bomb(input_geojson, bbox):
    gdf = gpd.read_file(input_geojson)
    
    # Column with category
    col_categories = 'agencia'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    #color_polygon['Bauru'] = '#e699ff'
    #color_polygon['Campinas'] = '#4da6ff'

    # Create Map
    m = folium.Map(
        location=[-22.545968889465207, -49.56185387118866],
        zoom_start=6,
        min_zoom=6,
        max_zoom=11,
        max_bounds=True,
        min_lon = bbox['min_lon']*(101/100),
        max_lon = bbox['max_lon']*(99/100),        
        min_lat = bbox['min_lat']*(101/100),
        max_lat = bbox['max_lat']*(99/100),
        #zoom_delta=0.1,
    )
    folium.GeoJson(
        gdf,
        name='CETESB',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'agencia'],
            aliases=['Munícipio', 'Agência Ambiental'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )        
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
bbox = {
    'max_lat': 0,
    'max_lon': 0,
    'min_lat': 0,
    'min_lon': 0,
}
m = map_bomb(os.path.join('data', 'shps', 'sp_cetesb.geojson'), bbox)

# Map with Bounds
bbox = {
    'max_lat': m.get_bounds()[1][0],
    'min_lat': m.get_bounds()[0][0],
    'max_lon': m.get_bounds()[1][1],
    'min_lon': m.get_bounds()[0][1],
}
m = map_bomb(os.path.join('data', 'shps', 'sp_cetesb.geojson'), bbox)

# Results
print(bbox)
m

In [ ]:
def add_script(map_folium):
    """
    """
    import folium
    import branca as bc
    
    # Header to Add
    head = """
    {% macro header(this, kwargs) %}
    <script>
    function toggleSeeMore() {
    if(document.getElementById("textarea").style.display == 'none') {
    document.getElementById("textarea").style.display = 'block';
    document.getElementById("seeMore").innerHTML = '...';
    }
    else {
    document.getElementById("textarea").style.display = 'none';
    document.getElementById("seeMore").innerHTML = 'Coordenadores Adjuntos';
    }
    }
    </script>
    {% endmacro %}
    """
    
    # Add Header
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(head)
    map_folium.get_root().add_child(macro)
    return map_folium

In [ ]:
m = add_script(m)
m

<br>

# Save Map

In [ ]:
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'cetesb_map.html'))
m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'cetesb_map.html'))